# <center> ⚡ ClearML + Lightning в CV: учим GAN 🛠

Вам необходимо **сдать файл** (или несколько файлов) с расширением `любое_имя.py` и **ссылку** на результат эксперимента в `ClearML`:

**Основное задание (5 баллов):**

1. Структурировать код с использованием `PyTorch Lightning`:

* Создать класс, наследующий от `LightningModule`, который реализует GAN.
* Разбить код на методы: `training_step, configure_optimizers, validation_step` (если требуется) и т. д.

2. Создать `LightningDataModule`:
* Реализовать методы `prepare_data(), setup(stage), train_dataloader()` (и, по желанию, `val_dataloader()`) для загрузки датасета MNIST.

3. Интегрировать `ClearML`:
* Проверять вначале работы скрипта наличие всех необходимых для работы `ClearML credentials`, при необходимости просить их ввести.
* Логировать гиперпараметры
* Логировать метрики (например, потери генератора и дискриминатора)
* Логировать промежуточные сгенерированные изображения, чтобы они отображались на вкладке `Debug samples` в UI.

4. Сохранение чекпоинтов:
* Использовать возможности `Lightning` для автоматического сохранения чекпоинтов модели.

**Дополнительное задание (2 балла):**
1. Добавьте считывание параметра `--epoch` при запуске файла на исполнение, который будет отвечать за количество эпох обучения (значение по умолчанию `10`).
2. Добавьте считывание параметра `--debug_samples_epoch` при запуске файла на исполнение, который будет отвечать за частоту логирования отладочных сэмплов: 1 - каждую эпоху, 2 - каждую вторую эпоху и.т.д (значение по умолчанию `1`).
Пример команды: `python любое_имя.py --epoch 20 --debug_samples_epoch 2`

**Задание со звездочкой 🌟 (1 балл):**

Создайте репозиторий на одном из хостингов: `GitHub, GitLab, GitVerse`
Загрузите на хостинг свое решение, добавив `README` файл с краткой информацией о проекте и его запуском
Прикрепите ссылку на репозиторий в текстовом поле.
Файл должен запускаться командой `python любое_имя.py` и отрабатывать до конца без падений.

Так же в текстовом поле приложите ссылку с результатами, которую можно создать в `ClearML` на странице эксперимента в `меню -> Share`.

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [8]:
# Гиперпараметры
batch_size = 64
lr = 0.0002
num_epochs = 10
noise_dim = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Подготовка датасета MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:01<00:00, 8256739.03it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 223295.72it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 2071931.98it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 9597243.71it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
# Определение генератора
class Generator(nn.Module):
    def __init__(self, noise_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Вход: вектор шума размера noise_dim
            nn.Linear(noise_dim, 256 * 7 * 7),
            nn.ReLU(True),
            nn.Unflatten(1, (256, 7, 7)),
            # Состояние: (256, 7, 7)
            nn.ConvTranspose2d(
                256, 128, kernel_size=4, stride=2, padding=1, bias=False
            ),  # -> (128, 14, 14)
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(
                128, 1, kernel_size=4, stride=2, padding=1, bias=False
            ),  # -> (1, 28, 28)
            nn.Tanh(),
        )

    def forward(self, input):
        return self.main(input)


# Определение дискриминатора
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Вход: изображение (1, 28, 28)
            nn.Conv2d(
                1, 64, kernel_size=4, stride=2, padding=1, bias=False
            ),  # -> (64, 14, 14)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(
                64, 128, kernel_size=4, stride=2, padding=1, bias=False
            ),  # -> (128, 7, 7)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1),
            nn.Sigmoid(),
        )

    def forward(self, input):
        return self.main(input)

In [9]:
# Создаем модели
netG = Generator(noise_dim).to(device)
netD = Discriminator().to(device)

# Определяем функцию потерь и оптимизаторы
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

# Метки
real_label = 1.0
fake_label = 0.0

In [10]:
# Основной цикл обучения
for epoch in range(num_epochs):
    for i, (data, _) in enumerate(dataloader):
        ############################
        #  Обновляем дискриминатор
        ############################
        netD.zero_grad()
        # Обучение на реальных изображениях
        real_images = data.to(device)
        b_size = real_images.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        output = netD(real_images).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # Обучение на фейковых изображениях
        noise = torch.randn(b_size, noise_dim, device=device)
        fake_images = netG(noise)
        label.fill_(fake_label)
        output = netD(fake_images.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        #  Обновляем генератор
        ############################
        netG.zero_grad()
        label.fill_(
            real_label
        )  # Для генератора "фейковые" метки считаются как реальные
        output = netD(fake_images).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        if i % 300 == 0:
            print(
                "[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(dataloader),
                    errD.item(),
                    errG.item(),
                    D_x,
                    D_G_z1,
                    D_G_z2,
                )
            )

    # Сохраняем сэмплы генератора каждые 2 эпохи
    if epoch % 2 == 0:
        with torch.no_grad():
            fixed_noise = torch.randn(64, noise_dim, device=device)
            fake = netG(fixed_noise).detach().cpu()
            os.makedirs("output", exist_ok=True)
            torchvision.utils.save_image(
                fake, f"output/fake_samples_epoch_{epoch}.png", normalize=True
            )

[0/10][0/938]	Loss_D: 1.3175	Loss_G: 0.8938	D(x): 0.5733	D(G(z)): 0.5213 / 0.4172
[0/10][300/938]	Loss_D: 0.0021	Loss_G: 7.1663	D(x): 0.9989	D(G(z)): 0.0009 / 0.0008
[0/10][600/938]	Loss_D: 0.0007	Loss_G: 8.0991	D(x): 0.9996	D(G(z)): 0.0003 / 0.0003
[0/10][900/938]	Loss_D: 0.0242	Loss_G: 4.8106	D(x): 0.9880	D(G(z)): 0.0115 / 0.0091
[1/10][0/938]	Loss_D: 0.0167	Loss_G: 5.3511	D(x): 0.9897	D(G(z)): 0.0059 / 0.0050
[1/10][300/938]	Loss_D: 0.5373	Loss_G: 2.5472	D(x): 0.8024	D(G(z)): 0.2376 / 0.1044
[1/10][600/938]	Loss_D: 0.6278	Loss_G: 2.3498	D(x): 0.8830	D(G(z)): 0.3795 / 0.1055
[1/10][900/938]	Loss_D: 0.3221	Loss_G: 2.5783	D(x): 0.8929	D(G(z)): 0.1707 / 0.0955
[2/10][0/938]	Loss_D: 0.3158	Loss_G: 2.7685	D(x): 0.9203	D(G(z)): 0.1878 / 0.0816
[2/10][300/938]	Loss_D: 0.4891	Loss_G: 3.7518	D(x): 0.9436	D(G(z)): 0.3114 / 0.0344
[2/10][600/938]	Loss_D: 0.3738	Loss_G: 1.5768	D(x): 0.7862	D(G(z)): 0.0808 / 0.2451
[2/10][900/938]	Loss_D: 0.4187	Loss_G: 1.9415	D(x): 0.8275	D(G(z)): 0.1719 / 0.181

In [7]:
# Сохраняем финальные веса генератора
torch.save(netG.state_dict(), "netG_final.pth")

# Запись файлов для сдачи на Stepik

In [ ]:
!pip install clearml lightning -q

In [11]:
%%writefile train.py
#!/usr/bin/env python


Writing example.py


In [ ]:
# Проверка
!python train.py

In [ ]:
# Проверка с доп.заданием
!python train.py --epoch 20 --debug_samples_epoch 2